# Demo: refactored implementation of $\Omega$

This notebook gives an example of the new implementation of $\Omega$. The aim of the implementation has so far been to make as few alterations as possible to the existing codebase, while still enabling the user considerably more flexibility in the definition of $\Omega$. 

## Math Assumptions

The primary mathematical assumption behind the new implementation is that $\Omega(z)$ is always expressible as a function of the partition vector of $z$. This is the same assumption that we were using throughout our previous version. The aim of the new implementation is that special cases can be handled with significantly more peformant code. 

In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using StatsBase

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [3]:
n = 20
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)
μ = mean(ϑ)
kmax = 4;

## The `IntensityFunction` struct

Now we're ready to construct $\Omega$. In the new idiom, $\Omega$ is an `IntensityFunction`, which contains four fields. 
Let's look at two examples, which are both implemented in `src/omega.jl`. 

```julia
function partitionIntensityFunction(ω, kmax)
    range      = partitionsUpTo(kmax)
    P          = partitionize
    aggregator = identity
    return IntensityFunction(ω, P, range, aggregator)
end
```

This `IntensityFunction` corresponds to the highly general `partitionize`-based framework we were using previously. Here are its fields: 

<dl>
    <dt> ω </dt> <dd> A parameterized function whose valid inputs are the elements of `range`. </dd>
    <dt> <code>P</code> </dt> <dd> A function that maps group label vectors (i.e. subvectors of $Z$) to <i>feature</i> vectors. <code>partitionize()</code> is an example, as is the polyadic $\delta$-function. </dd>
    <dt> <code>range</code> </dt> <dd> The set of all possible values of <code>P</code>. Also assumed to be the domain of the function $\omega$. This nomenclature is perhaps confusing and may be revised.</dd>
    <dt> <code>aggregator</code> </dt> <dd> A function that maps partition vectors (of the kind returned by <code>partitionize()</code> to elements of <code>range</code>. This is included for technical purposes related to the calculation of the volume term in modularity. It would be desirable to deprecate it if possible.  </dd>
</dl>

So, the code above creates an intensity function that operates on subvectors of $Z$ by first `partitionize()`ing them and then applying the function $\omega$. No aggregation is needed.  

On the other hand, here's an alternative that corresponds to the all-or-nothing cut. In this case, the feature map $P$ computes two entries: whether or not all entries of $z$ are the same, and the length of $z$. The `range` gives all possible results (up to size `kmax`). The aggregator takes a partition vector and returns a feature. The specified function $\omega$ should then operate on the features. 

```julia
function allOrNothingIntensityFunction(ω, kmax)
    range      = [(1.0*x, y) for x = 0:1 for y = 1:kmax]
    P          = z->(all(z[1] .== z), length(z))
    aggregator = p->(length(p) == 1, sum(p))
    return IntensityFunction(ω, P, range, aggregator)
end
```
Ok, let's try it out: 



In [4]:
function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

α = vcat(repeat([5.0], kmax), 0.2*(1:kmax))
Ω = partitionIntensityFunction(ω, kmax);

typeof(Ω)

IntensityFunction

In [5]:
H = sampleSBM(Z, ϑ, Ω;α=α, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((20,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
  E: Dict{Int64,Dict}
  D: Array{Int64}((20,)) [3, 6, 10, 3, 5, 2, 2, 7, 2, 6, 1, 7, 5, 5, 4, 7, 3, 8, 4, 11]


# TODO: check on inference and algorithms

# CONJECTURE: 

we have introduced a bug related to CutDiff_OneNode that is causing issues. Consider whether we should try to squash the bug or reimplement SuperNodeLouvain using more familiar functions such as cutDiff(). Major question will be whether this introduces additional expense (i.e. whether we gain from the penalty term used in the current code). 

I **think** that we do gain from storying the penalties -- should cut the number of calls to `Omega.P` by about half, which is certainly desirable. 




In [6]:
dataset = "contact-primary-school"
kmax_ = 6

H, Z = read_hypergraph_data(dataset,kmax_)

kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))

n = length(H.D)

# full, partition-based intensity function

function ω_1(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

Ω_1 = partitionIntensityFunction(ω_1, kmax);

# ---

# sum of exterior
function ω_2(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_2 = sumOfExteriorDegreesIntensityFunction(ω_2, kmax);


# sum of exterior
function ω_3(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_3 = allOrNothingIntensityFunction(ω_3, kmax);

# Quick Speed Tests

In [8]:
Z_dyadic = CliqueExpansionModularity(H);

In [67]:
α = zeros(2*kmax);

## General Partition-Based

In [33]:
α, f = learnParameters(H, Z_dyadic, Ω_1, α; ftol_abs = 1e-1)

([0.37235902432056794, 2.8678933926823387, 6.743124522082655, 7.683818220111279, 7.131635917772291, 7.6674009516602215, 0.9780918519042805, 1.1850954493288224, 1.4071329581303154, 1.6024599066543563], 207914.78210149752)

In [34]:
@time Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_1; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
 11.752488 seconds (69.77 M allocations: 3.277 GiB, 7.81% gc time)
10


## Sum of Exterior Degrees

In [55]:
α, f = learnParameters(H, Z_dyadic, Ω_2, α; ftol_abs = 1e-1)

([-3.55152055013857, -2.8419076483855767, -3.378465742859703, -5.064439262413201, -7.056383033632777, 46.37636938449891, -0.9987452635479722, 0.10181093867767275, 2.4044502901364346, 2.3743192790382204], 211828.87377020856)

In [56]:
@time Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_2; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  6.286571 seconds (51.77 M allocations: 1.994 GiB, 10.19% gc time)
9


## All-or-nothing

In [62]:
α, f = learnParameters(H, Z_dyadic, Ω_3, α; ftol_abs = 1)

([-5.166626719420295, -0.810169739949687, -3.354310832307315, -5.802713453197831, -8.846453973998866, 49.376131346487846, 1258.5179960504315, 55.483786753629765, 33.49339780609307, 1.3871703866306981], 230180.99969596814)

In [66]:
@time Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_3; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
  8.622355 seconds (71.69 M allocations: 2.754 GiB, 10.29% gc time)
8
